# <center> Capstone проект №1 <br> Идентификация пользователей по посещенным веб-страницам
# <center>Неделя 6.  Vowpal Wabbit

На этой неделе мы познакомимся с популярной библиотекой Vowpal Wabbit и попробуем ее на данных по посещению сайтов.

## Применение Vowpal Wabbit к данным по посещению сайтов

### Подготовка данных

**Далее посмотрим на Vowpal Wabbit в деле. Правда, в задаче нашего соревнования при бинарной классификации веб-сессий мы разницы не заметим – как по качеству, так и по скорости работы (хотя можете проверить), продемонстрируем всю резвость VW в задаче классификации на 400 классов. Исходные данные все те же самые, но выделено 400 пользователей, и решается задача их идентификации. Скачайте данные [отсюда](https://inclass.kaggle.com/c/identify-me-if-you-can4/data) – файлы `train_sessions_400users.csv` и `test_sessions_400users.csv`.**

In [1]:
import os
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.linear_model import LogisticRegression, SGDClassifier

In [4]:
# Поменяйте на свой путь к данным
PATH_TO_DATA = r'C:\Users\1\projects\ipynb\coursera\project\week6\data'

**Загрузим обучающую и тестовую выборки. Можете заметить, что тестовые сессии здесь по времени четко отделены от сессий в обучающей выборке. **

In [5]:
train_df_400 = pd.read_csv(os.path.join(PATH_TO_DATA,'train_sessions_400users.csv'), 
                           index_col='session_id')

In [6]:
test_df_400 = pd.read_csv(os.path.join(PATH_TO_DATA,'test_sessions_400users.csv'), 
                           index_col='session_id')

In [7]:
train_df_400.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,user_id
session_id,,,,,,,,,,,,,,,,,,,,,
1,23713,2014-03-24 15:22:40,23720.0,2014-03-24 15:22:48,23713.0,2014-03-24 15:22:48,23713.0,2014-03-24 15:22:54,23720.0,2014-03-24 15:22:54,...,2014-03-24 15:22:55,23713.0,2014-03-24 15:23:01,23713.0,2014-03-24 15:23:03,23713.0,2014-03-24 15:23:04,23713.0,2014-03-24 15:23:05,653
2,8726,2014-04-17 14:25:58,8725.0,2014-04-17 14:25:59,665.0,2014-04-17 14:25:59,8727.0,2014-04-17 14:25:59,45.0,2014-04-17 14:25:59,...,2014-04-17 14:26:01,45.0,2014-04-17 14:26:01,5320.0,2014-04-17 14:26:18,5320.0,2014-04-17 14:26:47,5320.0,2014-04-17 14:26:48,198
3,303,2014-03-21 10:12:24,19.0,2014-03-21 10:12:36,303.0,2014-03-21 10:12:54,303.0,2014-03-21 10:13:01,303.0,2014-03-21 10:13:24,...,2014-03-21 10:13:36,303.0,2014-03-21 10:13:54,309.0,2014-03-21 10:14:01,303.0,2014-03-21 10:14:06,303.0,2014-03-21 10:14:24,34
4,1359,2013-12-13 09:52:28,925.0,2013-12-13 09:54:34,1240.0,2013-12-13 09:54:34,1360.0,2013-12-13 09:54:34,1344.0,2013-12-13 09:54:34,...,2013-12-13 09:54:34,1346.0,2013-12-13 09:54:34,1345.0,2013-12-13 09:54:34,1344.0,2013-12-13 09:58:19,1345.0,2013-12-13 09:58:19,601
5,11,2013-11-26 12:35:29,85.0,2013-11-26 12:35:31,52.0,2013-11-26 12:35:31,85.0,2013-11-26 12:35:32,11.0,2013-11-26 12:35:32,...,2013-11-26 12:35:32,11.0,2013-11-26 12:37:03,85.0,2013-11-26 12:37:03,10.0,2013-11-26 12:37:03,85.0,2013-11-26 12:37:04,273


**Видим, что в обучающей выборке 182793 сессий, в тестовой – 46473, и сессии действительно принадлежат 400 различным пользователям.**

In [8]:
train_df_400.shape, test_df_400.shape, train_df_400['user_id'].nunique()

((182793, 21), (46473, 20), 400)

**Vowpal Wabbit любит, чтоб метки классов были распределены от 1 до K, где K – число классов в задаче классификации (в нашем случае – 400). Поэтому придется применить `LabelEncoder`, да еще и +1 потом добавить (`LabelEncoder` переводит метки в диапозон от 0 до K-1). Потом надо будет применить обратное преобразование.**

In [9]:
from sklearn import preprocessing

y = train_df_400['user_id']
class_encoder = preprocessing.LabelEncoder()
y_for_vw = class_encoder.fit_transform(y)+1

**Далее будем сравнивать VW с SGDClassifier и с логистической регрессией. Всем моделям этим нужна предобработка входных данных. Подготовьте для sklearn-моделей разреженные матрицы, как мы это делали в 5 части:**
- объедините обучающиую и тестовую выборки
- выберите только сайты (признаки от 'site1' до 'site10')
- замените пропуски на нули (сайты у нас нумеровались с 0)
- переведите в разреженный формат `csr_matrix`
- разбейте обратно на обучающую и тестовую части

In [10]:
sites = ['site' + str(i) for i in range(1, 11)]

In [11]:
train_test_df = pd.concat([train_df_400, test_df_400], sort=False)

In [12]:
train_test_df_sites = train_test_df[sites].fillna(0).astype('int')

In [15]:
def to_sparse_matrix(X, site_dict):
    M = max(site_dict.values())
    row_ind = []
    col_ind = []
    data = []
    i, j = 0, 0
    for line in X.values:
        for e in line:
            if e != 0:
                data.append(1)
                row_ind.append(i)
                col_ind.append(e-1)
        i += 1
    return csr_matrix((data, (row_ind, col_ind)), shape=(i, M))

In [17]:
import collections
import pickle
import numpy as np
from scipy.sparse import csr_matrix

def load_save_result(file_name):
    with open(file_name, 'rb') as f:
        return pickle.load(f)
    
def create_sparse_matr(X):
    site_dic = load_save_result(os.path.join(PATH_TO_DATA, 'site_dic.pkl'))
    nrows = X.shape[0]
    ncols = len(site_dic)
    result = np.zeros((nrows, ncols), dtype=np.uint8)

    for i in range(nrows):
        sites = X.iloc[i, :].values
        site_counter = collections.Counter(sites)
        for site_id, c in site_counter.most_common():
            result[i, site_id-1] = c
                            
    return csr_matrix(result, shape=(nrows, ncols), dtype=np.uint8)

In [19]:
%%time
train_test_sparse = create_sparse_matr(train_test_df_sites)

Wall time: 59.1 s


In [20]:
''' ВАШ КОД ЗДЕСЬ '''
X_train_sparse = train_test_sparse[:train_df_400.shape[0]]
X_test_sparse = train_test_sparse[train_df_400.shape[0]:]
y = train_df_400['user_id']

### 2.2. Валидация по отложенной выборке

**Выделим обучающую (70%) и отложенную (30%) части исходной обучающей выборки. Данные не перемешиваем, учитываем, что сессии отсортированы по времени.**

In [21]:
train_share = int(.7 * train_df_400.shape[0])
train_df_part = train_df_400[sites].iloc[:train_share, :]
valid_df = train_df_400[sites].iloc[train_share:, :]
X_train_part_sparse = X_train_sparse[:train_share, :]
X_valid_sparse = X_train_sparse[train_share:, :]

In [22]:
y_train_part = y[:train_share]
y_valid = y[train_share:]
y_train_part_for_vw = y_for_vw[:train_share]
y_valid_for_vw = y_for_vw[train_share:]

**Реализуйте функцию, `arrays_to_vw`, переводящую обучающую выборку в формат Vowpal Wabbit.**

Вход:
 - X – матрица `NumPy` (обучающая выборка)
 - y (необяз.) - вектор ответов (`NumPy`). Необязателен, поскольку тестовую матрицу будем обрабатывать этой же функцией
 - train – флаг, True в случае обучающей выборки, False – в случае тестовой выборки
 - out_file – путь к файлу .vw, в который будет произведена запись
 
Детали:
- надо пройтись по всем строкам матрицы `X` и записать через пробел все значения, предварительно добавив вперед нужную метку класса из вектора `y` и знак-разделитель `|`
- в тестовой выборке на месте меток целевого класса можно писать произвольные, допустим, 1

In [23]:
def arrays_to_vw(X, y=None, train=True, out_file='tmp.vw'):
    X = X.fillna(0).astype('int')
    with open(os.path.join(PATH_TO_DATA, out_file), "w") as outfile:  
        for i in range(0, X.shape[0]):
            result = ''
            if train:
                label = int(y[i])
            else:
                label = 1

            result = result + str(label) + ' | '

            for j in range(0, X.shape[1]):                
                value = X.iloc[i, j]

                result = result + str(value) + ' '

            outfile.write(result + "\n")

**Примените написанную функцию к части обучащей выборки `(train_df_part, y_train_part_for_vw)`, к отложенной выборке `(valid_df, y_valid_for_vw)`, ко всей обучающей выборке и ко всей тестовой выборке. Обратите внимание, что на вход наш метод принимает именно матрицы и вектора `NumPy`.**

In [24]:
%%time
# будет 4 вызова
arrays_to_vw(train_df_part, y_train_part_for_vw, train=True, out_file='train_part.vw')
arrays_to_vw(valid_df, y_valid_for_vw, train=True, out_file='valid.vw')
arrays_to_vw(train_df_400[sites], y=np.hstack((y_train_part_for_vw, y_valid_for_vw)), train=True, out_file='train.vw')
arrays_to_vw(test_df_400[sites], y=None, train=False, out_file='test.vw')

Wall time: 26.3 s


**Результат должен получиться таким.**

In [90]:
!head -3 $PATH_TO_DATA/train_part.vw

'head' is not recognized as an internal or external command,
operable program or batch file.


In [91]:
!head -3  $PATH_TO_DATA/valid.vw

'head' is not recognized as an internal or external command,
operable program or batch file.


In [92]:
!head -3 $PATH_TO_DATA/test.vw

'head' is not recognized as an internal or external command,
operable program or batch file.


**Обучите модель Vowpal Wabbitна выборке `train_part.vw`. Укажите, что решается задача классификации с 400 классами (`--oaa`), сделайте 3 прохода по выборке (`--passes`). Задайте некоторый кэш-файл (`--cache_file`, можно просто указать флаг `-c`), так VW будет быстрее делать все следующие после первого проходы по выборке (прошлый кэш-файл удаляется с помощью аргумента `-k`). Также укажите значение параметра `b`=26. Это число бит, используемых для хэширования, в данном случае нужно больше, чем 18 по умолчанию. Наконец, укажите `random_seed`=17. Остальные параметры пока не меняйте, далее уже в свободном режиме соревнования можете попробовать другие функции потерь.**

In [25]:
train_part_vw = os.path.join(PATH_TO_DATA, 'train_part.vw')
valid_vw = os.path.join(PATH_TO_DATA, 'valid.vw')
train_vw = os.path.join(PATH_TO_DATA, 'train.vw')
test_vw = os.path.join(PATH_TO_DATA, 'test.vw')
model = os.path.join(PATH_TO_DATA, 'vw_model.vw')
pred = os.path.join(PATH_TO_DATA, 'vw_pred.csv')

In [26]:
%%time
!vw $train_part_vw -f $model --oaa 400 --passes 3 --cache_file 'cache_file.txt' -k --random_seed 17 -b 26

Wall time: 26.3 s


final_regressor = C:\Users\1\projects\ipynb\coursera\project\week6\data\vw_model.vw
Num weight bits = 26
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = 'cache_file.txt'
Reading datafile = C:\Users\1\projects\ipynb\coursera\project\week6\data\train_part.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0      262        1       11
1.000000 1.000000            2            2.0       82      262       11
1.000000 1.000000            4            4.0      241      262       11
1.000000 1.000000            8            8.0      352      262       11
1.000000 1.000000           16           16.0      135       16       11
1.000000 1.000000           32           32.0       71      112       11
0.968750 0.937500           64           64.0      358      231       11
0.976563 0.984375       

**Запишите прогнозы на выборке *valid.vw* в *vw_valid_pred.csv*.**

In [27]:
vw_valid_pred = os.path.join(PATH_TO_DATA, 'vw_valid_pred.csv')

In [28]:
%%time
!vw -i $model -t -d $valid_vw -p $vw_valid_pred

Wall time: 684 ms


only testing
predictions = C:\Users\1\projects\ipynb\coursera\project\week6\data\vw_valid_pred.csv
Num weight bits = 26
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = C:\Users\1\projects\ipynb\coursera\project\week6\data\valid.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0        4      188       11
1.000000 1.000000            2            2.0      160      220       11
0.750000 0.500000            4            4.0      143      143       11
0.750000 0.750000            8            8.0      247      247       11
0.687500 0.625000           16           16.0      341       30       11
0.593750 0.500000           32           32.0      237      237       11
0.609375 0.625000           64           64.0      178      178       11
0.640625 0.671875          128          128.0      132      22

**Считайте прогнозы *kaggle_data/vw_valid_pred.csv*  из файла и посмотрите на долю правильных ответов на отложенной части.**

In [29]:
y_vw_valid_pred = pd.read_csv(vw_valid_pred, header=None)
y_vw_valid_pred.iloc[:, 0]

0        188
1        220
2        364
3        143
4        132
5        350
6          8
7        247
8         25
9        219
10        74
11       347
12       394
13       370
14       304
15        30
16       125
17       326
18       338
19       328
20        59
21       189
22       358
23        72
24        67
25       237
26       278
27       326
28       313
29       201
        ... 
54808    188
54809     37
54810    126
54811     86
54812    318
54813    380
54814     83
54815    313
54816     92
54817    113
54818    126
54819    389
54820    324
54821    355
54822    354
54823     55
54824    162
54825    231
54826     49
54827    233
54828     21
54829    220
54830    364
54831    147
54832    202
54833     74
54834    153
54835    118
54836    318
54837    125
Name: 0, Length: 54838, dtype: int64

**Теперь обучите `SGDClassifier` (3 прохода по выборке, логистическая функция потерь) и `LogisticRegression` на 70% разреженной обучающей выборки – `(X_train_part_sparse, y_train_part)`, сделайте прогноз для отложенной выборки `(X_valid_sparse, y_valid)` и посчитайте доли верных ответов. Логистическая регрессия будет обучаться не быстро (у меня – 4 минуты) – это нормально. Укажите везде `random_state`=17, `n_jobs`=-1. Для `SGDClassifier` также укажите `max_iter=3`.**

In [30]:
logit = LogisticRegression(random_state=17, n_jobs=-1)
sgd_logit = SGDClassifier(loss='log', max_iter=3, random_state=17, n_jobs=-1)

In [31]:
%%time
logit.fit(X_train_part_sparse, y_train_part)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1297: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 12.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


Wall time: 17min 48s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn', n_jobs=-1,
          penalty='l2', random_state=17, solver='warn', tol=0.0001,
          verbose=0, warm_start=False)

In [32]:
%%time
sgd_logit.fit(X_train_part_sparse, y_train_part)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


Wall time: 9.68 s


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=3,
       n_iter=None, n_iter_no_change=5, n_jobs=-1, penalty='l2',
       power_t=0.5, random_state=17, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False)

**<font color='red'>Вопрос 1. </font> Посчитайте долю правильных ответов на отложенной выборке для Vowpal Wabbit, округлите до 3 знаков после запятой.**

**<font color='red'>Вопрос 2. </font> Посчитайте долю правильных ответов на отложенной выборке для SGD, округлите до 3 знаков после запятой.**

**<font color='red'>Вопрос 3. </font> Посчитайте долю правильных ответов на отложенной выборке для логистической регрессии, округлите до 3 знаков после запятой.**

In [182]:
class_encoder.inverse_transform(y_vw_valid_pred.iloc[:, 0]-1)
#y_valid

array([474, 559, 928, ..., 305, 813, 317], dtype=int64)

In [157]:
y_vw_valid_pred.head()

,0
0,4
1,200
2,308
3,143
4,345


In [33]:
from sklearn.metrics import accuracy_score
vw_valid_acc = accuracy_score(y_valid, class_encoder.inverse_transform(y_vw_valid_pred.iloc[:, 0]-1))
sgd_valid_acc = accuracy_score(y_valid, sgd_logit.predict(X_valid_sparse))
logit_valid_acc = accuracy_score(y_valid, logit.predict(X_valid_sparse))

In [34]:
def write_answer_to_file(answer, file_address):
    with open(file_address, 'w') as out_f:
        out_f.write(str(answer))

In [35]:
write_answer_to_file(round(vw_valid_acc, 3), 'answer6_1.txt')
write_answer_to_file(round(sgd_valid_acc, 3), 'answer6_2.txt')
write_answer_to_file(round(logit_valid_acc, 3), 'answer6_3.txt')

### 2.3. Валидация по тестовой выборке (Public Leaderboard)

**Обучите модель VW с теми же параметрами на всей обучающей выборке – *train.vw*.**

In [130]:
%%time
!vw $train_vw -f $model --oaa 400 --passes 3 -c --random_seed 17 -b 26

Wall time: 1min 10s


final_regressor = capstone_user_identification\vw_model.vw
Num weight bits = 26
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = capstone_user_identification\train.vw.cache
Reading datafile = capstone_user_identification\train.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0      262        1       11
1.000000 1.000000            2            2.0       82      262       11
1.000000 1.000000            4            4.0      241      262       11
1.000000 1.000000            8            8.0      352      262       11
1.000000 1.000000           16           16.0      135       16       11
1.000000 1.000000           32           32.0       71      112       11
0.968750 0.937500           64           64.0      358      231       11
0.976563 0.984375          128          128.0      3

**Сделайте прогноз для тестовой выборки.**

In [131]:
vw_test_pred = os.path.join(PATH_TO_DATA, 'vw_test_pred.csv')

In [132]:
%%time
!vw -i $model -t -d $test_vw -p $vw_test_pred

Wall time: 1.11 s


only testing
predictions = capstone_user_identification\vw_test_pred.csv
Num weight bits = 26
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = capstone_user_identification\test.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0        1      207       11
1.000000 1.000000            2            2.0        1       21       11
1.000000 1.000000            4            4.0        1      265       11
1.000000 1.000000            8            8.0        1      137       11
1.000000 1.000000           16           16.0        1      273       11
1.000000 1.000000           32           32.0        1      265       11
1.000000 1.000000           64           64.0        1      298       11
1.000000 1.000000          128          128.0        1       85       11
1.000000 1.000000          256          2

**Запишите прогноз в файл, примените обратное преобразование меток (был LabelEncoder и потом +1 в меткам) и отправьте решение на Kaggle.**

In [112]:
def write_to_submission_file(predicted_labels, out_file,
                             target='user_id', index_label="session_id"):
    # turn predictions into data frame and save as csv file
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [133]:
y_vw_test_pred = pd.read_csv(vw_test_pred, header=None)

vw_pred = class_encoder.inverse_transform(y_vw_test_pred.iloc[:, 0]-1)

In [134]:
write_to_submission_file(vw_pred, os.path.join(PATH_TO_DATA, 'vw_400_users.csv'))

**Сделайте то же самое для SGD и логистической регрессии. Тут уже ждать обучение логистической регрессии совсем скучно (заново запускать тетрадку вам не захочется), но давайте дождемся.**

In [109]:
%%time
logit.fit(X_train_sparse, y)

C:\Python36_x64\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Python36_x64\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Python36_x64\lib\site-packages\sklearn\linear_model\logistic.py:1297: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


Wall time: 36min 48s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn', n_jobs=-1,
          penalty='l2', random_state=17, solver='warn', tol=0.0001,
          verbose=0, warm_start=False)

In [115]:
%%time
sgd_logit.fit(X_train_sparse, y)

Wall time: 9.98 s


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=3,
       n_iter=None, n_iter_no_change=5, n_jobs=-1, penalty='l2',
       power_t=0.5, random_state=17, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False)

In [116]:
logit_test_pred = logit.predict(X_test_sparse)
sgd_logit_test_pred = sgd_logit.predict(X_test_sparse)

In [118]:
write_to_submission_file(logit_test_pred, 
                         os.path.join(PATH_TO_DATA, 'logit_400_users.csv'))
write_to_submission_file(sgd_logit_test_pred, 
                         os.path.join(PATH_TO_DATA, 'sgd_400_users.csv'))

Посмотрим на доли правильных ответов на публичной части (public leaderboard) тестовой выборки [этого](https://inclass.kaggle.com/c/identify-me-if-you-can4) соревнования.

**<font color='red'>Вопрос 4. </font> Какова доля правильных ответов на публичной части тестовой выборки (public leaderboard)  для Vowpal Wabbit?**

**<font color='red'>Вопрос 5. </font> Какова доля правильных ответов на публичной части тестовой выборки (public leaderboard)  для SGD?**

**<font color='red'>Вопрос 6. </font> Какова доля правильных ответов на публичной части тестовой выборки (public leaderboard)  для логистической регрессии?**


In [135]:
vw_lb_score, sgd_lb_score, logit_lb_score = 0.18204, 0.17438, 0.20059

write_answer_to_file(round(vw_lb_score, 3), 'answer6_4.txt')
write_answer_to_file(round(0.17438, 3), 'answer6_5.txt')
write_answer_to_file(round(logit_lb_score, 3), 'answer6_6.txt')

**В заключение по заданию:**
- Про соотношение качества классификации и скорости обучения VW, SGD и logit выводы предлагается сделать самостоятельно
- Пожалуй, задача классификации на 400 классов (идентификация 400 пользователей) решается недостаточно хорошо при честном отделении по времени тестовой выборки от обучающей. Далее мы будем соревноваться в идентификации одного пользователя (Элис) – [вот](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) соревнование, в котором предлагается поучаствовать. Не перепутайте! 

**Удачи!**